# CSVファイル出力マニュアル ( python )

## 1. データベースへの接続

はじめに，sqlite3の実行とcsv出力に必要なモジュールをインポートします．  
Pythonには標準でSQLiteがインストールされています．

In [36]:
import sqlite3
import csv

今回，データベースの情報は「trial.sqlite3」というファイルに全て格納されているとします．  
( 実際は，配布されたデータベースファイルのファイル名に改めて実行してください．)

まず，以下のようにしてデータベースに接続します．

In [37]:
# Google Colaboratoryで作業する場合に実行してください．  
# データの読み込み方が不明な場合は，別途配布している「GoogleColabにおけるドライブのマウントの仕方」と「Google Drive上でのフォルダ構造の説明」のpdfファイルをご覧ください．
# from google.colab import drive
# drive.mount('/content/drive')

In [38]:
# データベースファイルのパスを渡してください
dbpath = '../data.sqlite3'

In [39]:
conn = sqlite3.connect(dbpath)

これで，データベースへの接続は完了です．  
※データベースが存在しない場合は，新規作成された上で接続されます．

## 2. データベースの観察

続いて，カーソルの取得をします．このモジュールでは，カーソルをイテレータとして扱います．  
カーソルを取得後にexecuteメソッドを用いると，SELECT文を実行することができます．

In [40]:
cur = conn.cursor()

ここでは、データの出力に必要となるデータベースの情報，特にテーブル構造について確認します．  
データの抽出や加工については，SQL回の講義資料等を参照ください．

In [41]:
# テーブル名の確認
cur.execute('select name from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('data',)
('goods',)


In [42]:
# カラム名(CREATE TABLE文)の確認
cur.execute('select sql from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)',)
('CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)',)


In [43]:
# 全情報の確認 (type, name, tbl_name, rootpage, sql)
cur.execute('select * from sqlite_master where type = "table"')
for row in cur.fetchall():
  print(row)

('table', 'data', 'data', 2, 'CREATE TABLE data(\n  "yy_mm_dd" TEXT,\n  "num_month" TEXT,\n  "store_id" TEXT,\n  "goods_id" TEXT,\n  "price" TEXT,\n  "units_sold_day" TEXT\n)')
('table', 'goods', 'goods', 12, 'CREATE TABLE goods(\n  "goods_name" TEXT,\n  "goods_id" TEXT,\n  "goods_genre_id" TEXT\n)')


In [44]:

cur.execute('select * from data')
count = 0
for row in cur.fetchall():
    print(row)
    count += 1
    if count > 10:
        break


('25-01-29', '-33', '43', '17718', '44943', '2')
('25-01-08', '-33', '32', '8648', '14949', '1')
('25-01-16', '-33', '53', '10168', '30158', '1')
('25-01-23', '-33', '31', '1005', '9922', '1')
('25-01-12', '-33', '27', '19861', '14988', '0')
('25-01-11', '-33', '26', '18473', '19768', '1')
('25-01-09', '-33', '51', '6096', '170492', '1')
('25-01-04', '-33', '7', '13902', '29793', '1')
('25-01-10', '-33', '23', '17213', '15099', '1')
('25-01-20', '-33', '47', '15105', '129142', '1')
('25-01-18', '-33', '1', '4367', '25964', '2')


In [45]:

cur.execute('select * from goods')
count = 0
for row in cur.fetchall():
    print(row)
    count += 1
    if count > 10:
        break


('! POWER IN glamor (PLAST.) D', '1', '41')
('! ABBYY FineReader 12 Professional Edition Full [PC, Digital Version]', '2', '77')
('*** In the glory (UNV) D', '3', '41')
('*** BLUE WAVE (Univ) D', '4', '41')
('*** BOX (GLASS) D', '5', '41')
('*** NEW American Graffiti (UNI) D', '6', '41')
('*** shot on goal (UNI) D', '7', '41')
('*** shot on goal-2 (UNI) D', '8', '41')
('*** Tea with Mussolini D', '9', '41')
('*** SHUGARLENDSKY EXPRESS (UNI) D', '10', '41')
('* BEYOND DEATH D', '11', '41')


In [49]:
cur.execute('select * from data')
data = cur.fetchall()

cur.execute('select * from goods')
goods = cur.fetchall()

In [65]:
# goods と data を

sql = """
SELECT 
    goods.goods_genre_id,
    AVG(CAST(data.price AS FLOAT)) as avg_price
FROM 
    data
LEFT OUTER JOIN 
    goods
ON 
    data.goods_id = goods.goods_id
GROUP BY 
    goods.goods_genre_id
ORDER BY
    CAST(goods.goods_genre_id AS INTEGER) ASC
"""


print(sql)

cur.execute(sql)
count = 0
for row in cur.fetchall():
    print(row)



SELECT 
    goods.goods_genre_id,
    AVG(CAST(data.price AS FLOAT)) as avg_price
FROM 
    data
LEFT OUTER JOIN 
    goods
ON 
    data.goods_id = goods.goods_id
GROUP BY 
    goods.goods_genre_id
ORDER BY
    CAST(goods.goods_genre_id AS INTEGER) ASC

('3', 168482.4)
('4', 47476.5)
('5', 129025.0)
('6', 39211.333333333336)
('7', 158876.2)
('12', 1381663.0)
('15', 1197012.5)
('16', 1315036.0)
('19', 29577.0)
('20', 154165.75)
('21', 248176.0)
('22', 52100.066666666666)
('23', 167440.53846153847)
('24', 164675.7037037037)
('26', 71894.57142857143)
('29', 63498.39473684211)
('30', 129826.33333333333)
('31', 42573.60625)
('32', 41988.6)
('34', 120571.0)
('36', 222425.0)
('38', 42881.25)
('39', 108544.72222222222)
('41', 27312.9203187251)
('42', 96667.75)
('44', 28968.470588235294)
('46', 20584.5)
('50', 34330.857142857145)
('56', 26939.964285714286)
('58', 26590.416666666668)
('59', 92398.5)
('60', 45314.42857142857)
('61', 171138.33333333334)
('62', 79763.0)
('63', 67526.75)
('64', 445

In [67]:
# goodsのgenre_idの最小値と最大値を求めるsql文
sql_min_max = """
SELECT 
    MIN(CAST(goods_genre_id AS INTEGER)) as min_genre_id,
    MAX(CAST(goods_genre_id AS INTEGER)) as max_genre_id
FROM 
    goods
"""

cur.execute(sql_min_max)
for row in cur.fetchall():
    print(row)


(1, 84)


## 3. データの出力 (csv形式)

fetchoneメソッドで1件ずつ取得する場合

In [64]:
cur.execute(sql)
print(cur.fetchone())  # 1レコード目の取得
print(cur.fetchone())  # 2レコード目の取得


(1, 84)
None


fetchallメソッドで全件を取得する場合 ※実行時間が長いことがあります

In [ ]:
cur.execute('select * from <テーブル名>')
for row in cur.fetchall():
    print(row)


csv出力 ( [out] は任意のファイル名)

In [62]:
# テーブルから特定の列を取得
cur.execute(sql)
with open('out.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow([i[0] for i in cur.description])
    csv_writer.writerows(cur)


In [ ]:
# テーブルから全ての列を取得
cur.execute('select * from <テーブル名>')
with open('out.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow([i[0] for i in cur.description])
    csv_writer.writerows(cur)


In [68]:
# 処理を確定
conn.commit()

左サイドバーのFilesに「out.csv」が出力されていることを確認してください．  
出力されたcsvファイルが確認出来たら，カーソルを閉じ，接続を閉じます．

In [69]:
cur.close()

In [70]:
conn.close()

## 【補足】出力の確認

In [71]:
import pandas as pd


In [72]:
pd.read_csv('out.csv', sep=',')

,goods_genre_id,avg_price
0,3,1.684824e+05
1,4,4.747650e+04
2,5,1.290250e+05
3,6,3.921133e+04
4,7,1.588762e+05
5,12,1.381663e+06
6,15,1.197012e+06
7,16,1.315036e+06
8,19,2.957700e+04
9,20,1.541658e+05


以上で，ライブラリを用いたcsv出力は完了です．